In [284]:
# Libraries
import pandas as pd
import networkx as nx
from tqdm import tqdm
import pickle # used to store the files
import itertools
from itertools import permutations
import folium # used to plot
import numpy as np
import decimal
from  more_itertools import unique_everseen
from collections import defaultdict
from tqdm import tqdm_notebook
import builtins

## 1. Prerequisites

## 1.1 Create Dataframes

Given the files we create a **DataFrame** for each of them. We will after convert these df into dictionaries.

### 1.1.1 Coordinates (Node information file)

In [285]:
coordinates = pd.read_csv(r"C:\Users\Luca\Desktop\WIP\ADM HW 5\data\USA-road-d.CAL.co", sep=' ', delimiter=None,
                      index_col=None, usecols=None, encoding='ISO-8859-1')

C:\Users\Luca\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [286]:
# combining two columns pandas
coordinates['combined']=coordinates['c'].astype(str)+''+coordinates['9th'].astype(str)

In [287]:
coordinates = coordinates.iloc[6:] # deleting the first 5 rows

In [288]:
# organizing pandas DF
coordinates.drop(['c', '9th', 'Challenge:', 'Shortest', 'Paths'], axis=1, inplace=True)

In [289]:
coordinates.columns = ['Latitude', 'Longitude', 'Id_Node']

In [290]:
coordinates = coordinates.reset_index(drop=True)

In [291]:
# Transform all str into int
coordinates[['Latitude', 'Longitude']] = coordinates[['Latitude', 'Longitude']].astype(int)

In [292]:
coordinates.head(5)

,Latitude,Longitude,Id_Node
0,-114315309,34133550,v1
1,-114223946,34176221,v2
2,-114307299,34148791,v3
3,-114318765,34138889,v4
4,-114347300,34042614,v5


### 1.1.2 Distance graph

In [273]:
distance = pd.read_csv(r"data\USA-road-d.CAL.gr", sep=' ', delimiter=None,
                      index_col=None, usecols=None, encoding='ISO-8859-1', error_bad_lines=False)

b'Skipping line 6: expected 7 fields, saw 8\n'


In [274]:
distance = distance.iloc[5:] # deleting the first 5 rows

In [275]:
# organizing pandas DF
distance.drop(['c','Challenge:', 'Shortest', 'Paths'], axis=1, inplace=True)

(Id_Node1, Id_Node2, d(Id_Node1,Id_Node2)), where d(x,y) is the physical distance between x and y.

In [276]:
distance.columns = ['Id_Node1', 'Id_Node2', 'Distance']

In [277]:
distance = distance.reset_index(drop=True)

In [278]:
# Transform all str into int
distance = distance.astype(int)

In [279]:
distance.head(5)

,Id_Node1,Id_Node2,Distance
0,1,1048577,456
1,1048577,1,456
2,2,1048578,2389
3,1048578,2,2389
4,3,1048579,358


### 1.1.3 Time travel graph

In [266]:
travel = pd.read_csv(r"data\USA-road-t.CAL.gr", sep=' ', delimiter=None,
                      index_col=None, usecols=None, encoding='ISO-8859-1', error_bad_lines = False)

b'Skipping line 6: expected 7 fields, saw 8\n'
C:\Users\Luca\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [267]:
travel = travel.iloc[5:] # deleting the first 5 rows

In [268]:
# organizing pandas DF
travel.drop(['c','Challenge:', 'Shortest', 'Paths'], axis=1, inplace=True)

(Node1, Node2, t(Id_Node1, Id_Node2))

In [269]:
travel.columns = ['Id_Node1', 'Id_Node2', 'Time distance']

In [270]:
travel = travel.reset_index(drop=True)

In [271]:
# Transform all str into int
travel = travel.astype(int)

In [272]:
travel.head(5)

,Id_Node1,Id_Node2,Time distance
0,1,1048577,1139
1,1048577,1,1139
2,2,1048578,5972
3,1048578,2,5972
4,3,1048579,895


## 1.2 Create data types for nodes and edges

In [ ]:
# Create a list of all nodes
nodes = list(range(1, 1890816))


In [ ]:
# Create a list of set with (Node_1, Node_2)
edges = []

In [ ]:
temp_travel = travel.drop(['Time distance'], axis=1)
for index, element in tqdm(temp_travel.iterrows()):
    edge = [] # couples of edges
    edge.extend([element[0], element[1]])
    edges.append(edge)

We have to find out if all verteces are bidirectional, to state that it's a bidirectional or directional graph. <br/>
Since the data downloaded talkes about Arcs instead of Edges we can assume that it's a directed graph

In [ ]:
with open('outfile', 'wb') as fp:
    pickle.dump(edges, fp)

In [ ]:
with open ('outfile', 'rb') as fp:
    edges = pickle.load(fp)

### 1.2.1 Create the adjacency dictionary

In [ ]:
adj_dic = {}

In [ ]:
for i in tqdm(edges):
    if i[0] not in adj_dic: # check if first node is a key already 
        
        adj_dic[i[0]] = []
        adj_dic[i[0]].append(i[1])
    
    else:
        
        if i[1] in adj_dic[i[0]]:
            continue
        else:
            adj_dic[i[0]].append(i[1])  # if True check if second node is in dictionary, if not append
    
    if i[1] not in adj_dic: # check if first node is a key already
        
        adj_dic[i[1]] = []
        adj_dic[i[1]].append(i[0])
    
    else:
        
        if i[0] in adj_dic[i[1]]:
            continue
        else:
            adj_dic[i[1]].append(i[0])  # if True check if second node is in dictionary, if not append

In [ ]:
# we saved this dictionary to a picke file so that we don't have to run the code each time
with open('adj_dic', 'wb') as fp:
    pickle.dump(adj_dic, fp)

In [ ]:
# Read the files
with open('adj_dic', 'rb') as fp:
    adj_dic = pickle.load(fp)

### 1.2.2 Weighted graph

#### Time 

In [ ]:
graph_trav_dist = defaultdict(list)
for row in tqdm_notebook(travel.iterrows()):
    if str(row[1]["Id_Node2"]) in adj_dic[str(row[1]["Id_Node1"])]:
        graph_trav_dist[str(row[1]["Id_Node1"])].append((str(row[1]["Id_Node2"]),int(row[1]["Time distance"])))
        

In [ ]:
# we saved this dictionary to a picke file so that we don't have to run the code each time
with open('adj_dic_travel', 'wb') as fp:
    pickle.dump(graph_trav_dist,fp)

In [351]:
with open('adj_dic_travel', 'rb') as fp:
    graph_trav_dist = pickle.load(fp)

#### Distance

In [293]:
graph_ph_dist = defaultdict(list)
for row in tqdm_notebook(distance.iterrows()):
    if int(row[1]["Id_Node2"]) in adj_dic[int(row[1]["Id_Node1"])]:
        graph_ph_dist[str(row[1]["Id_Node1"])].append((str(row[1]["Id_Node2"]),int(row[1]["Distance"])))

In [294]:
# we saved this dictionary to a picke file so that we don't have to run the code each time
with open('adj_dic_ph', 'wb') as fp:
    pickle.dump(graph_ph_dist,fp)

In [295]:
with open('adj_dic_ph', 'rb') as fp:
    graph_ph_dist = pickle.load(fp)

#### Network

In [ ]:
graph_net_dist = defaultdict(list)
for row in tqdm_notebook(distance.iterrows()):
    if int(row[1]["Id_Node2"]) in adj_dic[int(row[1]["Id_Node1"])]:
        graph_net_dist[int(row[1]["Id_Node1"])].append((int(row[1]["Id_Node2"]), 1)      

In [ ]:
# we saved this dictionary to a picke file so that we don't have to run the code each time
with open('adj_dic_net', 'wb') as fp:
    pickle.dump(graph_ph_dist,fp)

In [ ]:
with open('adj_dic_net', 'rb') as fp:
    graph_net_dist = pickle.load(fp)

---------------------------------

---------------------------------
# Functionalities


## Functionality 1 - Find the Neighbours!

In [322]:
v=int(input("Please type the id of the node you want to start with: "))
metric=input("Please type the metric you prefer. The options are 'distance' or 'time': ")
val=int(input("Please type the numerical value of the chosen metric: "))

# If the user choose spatial distance...

if (metric=="distance"):
    def recursive_function(v,val,the_list):
        df2 = distance[(distance.Id_Node1==v) & (distance.Distance <= val) & (~distance.Id_Node2.isin(the_list))]
        df2['dist_left'] = val-df2['Distance']
        the_list.extend(list(df2.Id_Node2))
        n = df2[['Id_Node2','dist_left']].shape[0]
        for i in range(n):
            recursive_function(df2.iloc[i,1],df2.iloc[i,3],the_list) #applying the search recursively
    def call_function(v,val):
        the_list=[]
        recursive_function(v,val,the_list)
        the_list=[v]+the_list
        neighbour=pd.DataFrame(the_list).reset_index(drop=True)
        neighbour.columns=['Neighbouring_nodes']
        return(neighbour) 
    
# If the user choose time...
elif(metric=="time"):
    def recursive_function(v,val,the_list):
        df2 = travel[(travel.Id_Node1==v) & (travel['Time distance'] <= val) & (~travel.Id_Node2.isin(the_list))]
        df2['dist_left'] = val-df2['Time distance']
        the_list.extend(list(df2.Id_Node2))
        n = df2[['Id_Node2','dist_left']].shape[0]
        for i in range(n):
            recursive_function(df2.iloc[i,1],df2.iloc[i,3],the_list) #applying the search recursively
    def call_function(v,val):
        the_list=[]
        recursive_function(v,val,the_list)
        the_list=[v]+the_list
        neighbour=pd.DataFrame(df2,the_list).reset_index(drop=True)
        neighbour.columns=['Neighbouring_nodes']
        return(neighbour) 
else: 
    print("Wrong option chosen")

# Check the code....


Please type the id of the node you want to start with: 2
Please type the metric you prefer. The options are 'distance' or 'time': distance
Please type the numerical value of the chosen metric: 6500


In [ ]:
neighbour=call_function(v,val)

Now that we have all the neighbours of the node we asked for we can plot it out

### Functionality 1. Plot

In [324]:
neighbour.head(5)

,Neighbouring_nodes
0,2
1,1048578
2,1050174
3,2462
4,2


In [325]:
# First let's create a list of nodes
n = neighbour['Neighbouring_nodes'].tolist()

In [326]:
# we then create a list of unique 
n = list(unique_everseen(n))

In [337]:
map1 = folium.Map(location = [34.133550,-114.315309], zoom_start = 6)

In [338]:
first = n.pop(0)

In [339]:
# We are going to use a different marker to show the initial node
longitude = str(coordinates.iloc[int(first)-1]['Longitude'])
longitude = decimal.Decimal(longitude[:2] + "." + longitude[2:])

latitude = str(coordinates.iloc[int(first)-1]['Latitude'])
latitude = decimal.Decimal(latitude[:4] + "." + latitude[4:])

folium.Marker(
    location=[longitude, latitude],
    popup='Timberline Lodge',
    icon=folium.Icon(color='red')
).add_to(map1)

In [340]:
coor = []
for node in n:
    
    longitude = str(coordinates.iloc[int(node)-1]['Longitude'])
    longitude = decimal.Decimal(longitude[:2] + "." + longitude[2:])
    
    latitude = str(coordinates.iloc[int(node)-1]['Latitude'])
    latitude = decimal.Decimal(latitude[:4] + "." + latitude[4:])
    
    folium.Marker([longitude, latitude]).add_to(map1)
    coor.append((longitude, latitude))


In [341]:
#folium.PolyLine(coor).add_to(map1)

*Zoom in to find nodes and edges*

In [342]:
map1

---------------------------------

## Functionality 2 - Find the smartest Network!

We used a generalized version of Dijkstra to resolve functionality 2. <br/>
How does it operate?

__*Functions used*__

In [358]:
# graph_net_dist --> Network Distance
# graph_ph_dist --> Physical Distance
# graph_trav_dist --> Travel Distance

Let's pick wich distance we want to use: <br/>
- Distance
- Time 
- Network

In [ ]:
metric = input("Please type the metric you prefer. The options are 'distance', 'time' or 'network': ")

In [ ]:
if (metric=="distance"):
    nodes = list(graph_ph_dist.keys())
    distances = graph_ph_dist
        
elif(metric=="time"):
    nodes = list(graph_ph_dist.keys())
    distances = graph_ph_dist

elif (metric=="network") : 
    nodes = list(graph_net_dist.keys())
    distances = graph_net_dist


In [387]:
set_of_nodes = input()

1 2 3


In [ ]:
set_of_nodes = set_of_nodes.split(" ") # we split the input nodes
perm = permutations(set_of_nodes, 2) # we create permutation of nodes

In [361]:
all_distances = {} # we'll use this to store the distances

for p in perm: # for every permutation
    unvisited = {node: None for node in nodes} #using None as +inf
    visited = {} # we initialize it as an empty dictionary
    current = int(p[0]) # p --> (Node1, Node2)
    end = int(p[1])
    currentDistance = 0
    unvisited[current] = currentDistance # unvisited --> {Node1: 0}

    while True:
        for neighbour, distance in distances[current]: # distance --> {Node1: [(neighbour, distance)...]}
            if neighbour not in unvisited: continue # go to next iteration

            newDistance = currentDistance + distance # we update the distance

            if unvisited[neighbour] is None or unvisited[neighbour] > newDistance: 
                unvisited[neighbour] = newDistance

        visited[current] = currentDistance
        del unvisited[current] # we updated the ditionary previously

        if not unvisited or current == end: 
            all_distances[p] = currentDistance
            break

        candidates = [node for node in unvisited.items() if node[1]]
        current, currentDistance = sorted(candidates, key = lambda x: x[1])[0]

('2', '1')
('2', '3')
('3', '1')
('3', '2')


In [362]:
candidates = []
for pair1 in all_distances:
    for pair2 in all_distances:
        if pair1[1] == pair2[0] and pair1[0] != pair2[1]:
            candidates.append((pair1, pair2))

In [371]:
candidates

[(('2', '3'), ('3', '1')), (('3', '2'), ('2', '1'))]

In [363]:
p = permutations(candidates, len(candidates)-1)

In [364]:
permu = [i for i in p]

In [365]:
res = []
for i in permu:
    
    for z in i:
        trues = []
        print(z)
        for j in range(len(candidates)-3):
            print(z[j])
            trues.append(z[j][1]==z[j+1][0])
        if all([l == True for l in trues] ):
            res.append(z)
            

(('2', '3'), ('3', '1'))
(('3', '2'), ('2', '1'))


In [366]:
distances = []
for tuple_ in res[:-1]:
    try:
        distances.append((all_distances[tuple_[0]] + all_distances[tuple_[1]]))
    except:
        pass

In [367]:
# the best result!
best = candidates[distances.index(min(distances))]

In [368]:
b = [i for sublist in best for i in sublist]

In [369]:
b = list(unique_everseen(b))

In [370]:
b

['2', '3', '1']

### Functionality 2. Plot

In [374]:
map = folium.Map(location = [34.133550,-114.315309], zoom_start = 6)

In [375]:
coor = []
for node in b:
    
    longitude = str(coordinates.iloc[int(node)-1]['Longitude'])
    longitude = decimal.Decimal(longitude[:2] + "." + longitude[2:])
    
    latitude = str(coordinates.iloc[int(node)-1]['Latitude'])
    latitude = decimal.Decimal(latitude[:4] + "." + latitude[4:])
    
    folium.Marker([longitude, latitude]).add_to(map)
    coor.append((longitude, latitude))


In [376]:
folium.PolyLine(coor).add_to(map)

In [377]:
map

## Functionality 3 - Shortest Ordered Route

We'll use Dijkstra algorithm to find the shortest paths

In [ ]:
def dijkstra(adj, start, end):

    pred = {x: x for x in adj } 
    dist = {x: float('inf') for x in adj }# All the distances to infinite
    dist[start] = 0 # First node has distance 0
    queue = []
    heapq.heappush(queue, [dist[start], start])# we insert the tuple (dist[start], start) from the start node

    while(queue):
        u = heapq.heappop(queue)  # u --> tuple [u_dist, u_id] 
        u_dist = u[0]
        u_id = u[1]
        if u_dist == dist[u_id]:
            if u_id == end: #If we have found the shortest path to the end we are done
                break
            for v in adj[u_id]: # adj --> {Node: [(Connected node, distance)...]}
                v_id = v[0]
                w_uv = v[1]
                if dist[u_id] +  w_uv < dist[v_id]: # If we find a shorter distance we can update it
                    dist[v_id] = dist[u_id] + w_uv
                    heapq.heappush(queue, [dist[v_id], v_id]) # heappush --> Pushes the value item onto the heap, maintaining the heap invariant.
                    pred[v_id] = u_id
    
    if dist[end] == float('inf'):# If there is no path
        stdout.write("There is no path between " + start + " and " + end)
    
    # We go backwards from the end to the start following the shortest path
    else:
        st = []
        node = end
        while(True):
            st.append(str(node))
            if(node == pred[node]):
                break
            node = pred[node]
        path = st[::-1] # we reverse the path list
        return path # we return the shortest path

In [401]:
node=input('Insert starting node ')
nodes_seq=list(input('Insert node sequence ').strip().split())
metric=input("Please type the metric you prefer. The options are 'distance', 'time' or 'network': ")
final_path=[]
nodes_seq.insert(0, node)

Insert starting node 72
Insert node sequence 138 120 101
Please type the metric you prefer. The options are 'distance', 'time' or 'network': distance


In [ ]:
if (metric=="distance"):
    
    g1 = dict(graph_ph_dist)
    for n in range(len(nodes_seq)-1):#We calculate the final path
        path=dijkstra(g1, nodes_seq[n], nodes_seq[n+1])
        for i in range(len(path)):
            final_path.append(path[i])
            
elif(metric=="time"):
    
    g2 = dict(graph_trav_dist)
    for n in range(len(nodes_seq)-1):#We calculate the final path
        path=dijkstra(g2, nodes_seq[n], nodes_seq[n+1])
        for i in range(len(path)):
            final_path.append(path[i])

elif (metric=="network") : 
    
    g3 = dict(graph_net_dist)
    for n in range(len(nodes_seq)-1):#We calculate the final path
        path=dijkstra(g3, nodes_seq[n], nodes_seq[n+1])
        for i in range(len(path)):
            final_path.append(path[i])
else:
    print("Error")

### Functionality 3. Plot

In [310]:
# we then create a list of unique nodes
n = builtins.map(int, final_path) 
n = list(unique_everseen(n))

In [311]:
map3 = folium.Map(location = [34.133550,-114.315309], zoom_start = 6)

In [312]:
first = n.pop(0) # --> starting point
last = n.pop(len(n)-1) # --> ending point

**Initial Node** (Red)

In [313]:
# We are going to use a different marker to show the initial node
longitude = str(coordinates.iloc[int(first)-1]['Longitude'])
longitude = decimal.Decimal(longitude[:2] + "." + longitude[2:])

latitude = str(coordinates.iloc[int(first)-1]['Latitude'])
latitude = decimal.Decimal(latitude[:4] + "." + latitude[4:])

folium.Marker(
    location=[longitude, latitude],
    popup='Timberline Lodge',
    icon=folium.Icon(color='red')
).add_to(map3)

**Ending Node** (Green)

In [314]:
# We are going to use a different marker to show the initial node
longitude = str(coordinates.iloc[int(last)-1]['Longitude'])
longitude = decimal.Decimal(longitude[:2] + "." + longitude[2:])

latitude = str(coordinates.iloc[int(last)-1]['Latitude'])
latitude = decimal.Decimal(latitude[:4] + "." + latitude[4:])

folium.Marker(
    location=[longitude, latitude],
    popup='Timberline Lodge',
    icon=folium.Icon(color='green')
).add_to(map3)

In [315]:
coor = []
for node in n:
    
    # We locate the coordinates of the node
    longitude = str(coordinates.iloc[int(node)-1]['Longitude']) 
    longitude = decimal.Decimal(longitude[:2] + "." + longitude[2:])
    
    latitude = str(coordinates.iloc[int(node)-1]['Latitude'])
    latitude = decimal.Decimal(latitude[:4] + "." + latitude[4:])
    
    # We add a marker for each node in the list n
    folium.Marker([longitude, latitude]).add_to(map3)
    coor.append((longitude, latitude))


In [316]:
folium.PolyLine(coor).add_to(map3)
map3